In [ ]:
AI Advatars

Use AI to evaluate an advertising creative against the most likely buyer personas. This colab walks you out through 
the step by step process so we can get ideas, customize them, and build your own client specific avatars

1. Configure Gemini AI to drive the avatars.
2. Star with a product description.
3. Gemini will find the best avatars to evaluate
4. Reliably move the avatars into Python structures
5. Use each avatar to:

a. Evaluate an existing image creative to get edit suggestions.
b. Prioritize personas to invest in custom campaigns
c. Get user journey insights into the persona
d. Create an audience targeting plan for each persona.
e. Generate keyword lists
f. Create an effective landing page template for each avatar.

1. Create summary sheet of each avatar for the campaign

This is advertising specific toolkit to get started with AI and advertising creatives. Key concepts you can leverage 
in your day to day are:

a. Examples of advertising prompts.
b. Practical means of moving data from AI to Python for processing.
c. Using Gemini for text and fir images in tandem.
d. Building a data structure that can be fed into a Google Ads Product.

Every step in this workbook builds on the last, so if we modify a step, remember to re-run the next steps in
the sequence. To help to tune the prompts each step is designed to be run as many times that need to get the 
output to where like it. Although can use this workbook without editing code, the prompts are intended to be customized 
and evolved to the product and needs 

Prompts to adjust fields and content.
Data Structures to add or remove avatars for example.
Models the Gemini Text model is better at text, use Gemini Vision Only for images.
Sections to add entirely new data for each avatar. 

Setting Expectations

1. This is an AI integration, things may format incorrectly, try re-running a step if it errors on parsing.
2. To see output may need to expand the cells.

Setup

Install Vertex AI SDK


In [ ]:
!pip install google-cloud-aiplatform --upgrade --user --quiet

In [ ]:
#Automatically restart kernel after installs so that your environment can access the new packages

import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Authenticating the notebook environment

If using Colab to run this notebook, uncomment the cell below and continue. If using Vertex AI Workbench, check out the setup instructions here (https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)

In [ ]:
from google.colab import auth

auth.authenticate_user()

Set the project ID and initialize Vertex AI SDK

Get PROJECT ID using gcloud

In [ ]:
#Define project informarion
PROJECT_ID = "gtech-kenjora" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

Import Libraries

In [ ]:
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part

AI Advatars

In [ ]:
#@title Add Product Description To Get Avatars
#@markdown This product description will be used to build the information

product = "mazda 3 hatchback" # @param {type:"string"}

In [ ]:
#@title Add an existing Creative to evaluate against avatars
#@markdown This image will be evaluated by the avatars
#@markdown Click on the folder icon on the left and upload an image, then paste the path here

from base64 import b64encode
from IPython.display import HTML, Markdown
from PIL import Image as PIL_Image

def embed_image(path_to_image):
    try:
       img = open(path_to_image, 'rb').read()
       return 'data:image/jpeg;base64,' + b64encode(img).decode()
    except FileNotFoundError:
       return ''

picture_url = '/content/creative.jpg' # @param {type:"string"}

try:
   picture = PIL_Image.open(picture_url)
except OSError:
   picture = None
   print('WARNING: No uploaded creative detected, some parts of this workbook ay not work.')

HTML(f'')

In [ ]:
#@title Set Up Imports And Helpers Plus Initialize Chat
#@markdown This just sets up common workbook functions and import necessary libraries.

import re
import json
from IPython.display import HTML, Markdown

RE_LIST = re.compile(r'


', re.DOTALL)
RE_DICT = re.compile(r'{.*}', re.DOTALL)
RE_HTML = re.compile(r'<.*>', re.DOTALL)

def response_to_list(response):
    return json.loads(RE_LIST.search(response.text).group(0))

def response_to_dict(response):
    try:
      return json.loads(RE_DICT.search(response.text).group(0))
    except json.JSONDecodeError:
      print('Parse JSON Error:', response.text)

def response_to_html(response):
    return RE_HTML.search(response.text).group(0)

model_vision = GenerativeModel("gemini-1.0-pro-vision")
#model_vision = genai.GenerativeModel('gemini-pro-vision')
model_text = GenerativeModel("gemini-1.0-pro")
#model_text = genai.GenerativeModel('gemini-pro')
chat = model_text.start_chat(history=[])


STEP 1: Get A List of AI Recommended Avatars

The output will be a python object, if get errors try adjusting the product description.

In [ ]:
prompt = f'Image you\'re an agency directory running an advertising campaign for the {product}, provide descriptive
persona names and explanations of why each is a fit.'

response = response_to_list(response)
history = chat.history
print(json.dumps(avatars, indent=2))


Step 2: Evaluate the creative against each avatar

The output will be added to your existing PYTHON object, if you get errors try adjusting the product description.

In [ ]:
if picture is None:
   print('Go back and upload an image to the colab, then run the creative step again')
else:
   target_picture = Image.load_from_file(picture_url)
   for avatar im avatars:
       print('Running:', avatar['persona'])
       prompt = f'Imagine you\'re a display marketing expert, evaluate the attached image against the persona {avatar["persona"]}, {avatar["description"]}. List things to keep, change and suggest three ideal 
       advertising images for {avatar["persona"]}:'
       response = model_vision.generate_content(
        contents = [f'Format your response as JSON dictionary with the keys: keep, change, sample.\n\n{prompt}', target_picture]
       )
       avatar['edits'] = response_to_dict(response)

print(json.dumps(avatars, indent = 2))

Step 3: Rank the avatars based on product fit

This demonstrates how to use personas in a single prompt to generate a ranked list. This creates a new object.

In [ ]:
table = '\n'.join(f'{avatar["persona"]}: {avatar["description"]}' for avatar in avatars)
prompt = f'Of the following personas, which one is most and least likely to purchase after seeing the attached advertisment, and why:\n\n{table}'
response = model_vision.generate_content(
    contents=[f'Format your response as JSON dictionary with three keys: most, least, reason.\n\n{prompt}', target_picture]
)
fit = response_to_dict(response)
print(json.dumps(fit, indent=2))

Step 4: Get Audience Suggestions for each avatar

Let Gemini determine which method of the targeting is the best fit for each avatar

In [ ]:
methods = [
  {
    "title": "Floodlight Targeting",
    "description": "Tracks user behavior across web, app, and ads using Google Marketing Platform's conversion tracking pixel to create tag-based audiences."
  },
  {
    "title": "Activity-based Targeting",
    "description": "Creates audiences based on campaign interactions or excludes users based on impression counts."
  },
  {
    "title": "YouTube User List Targeting",
    "description": "Creates YouTube remarketing lists based on interactions with your videos, ads, or channel."
  },
  {
    "title": "Customer Match Targeting",
    "description": "Uploads customer CSV for targeting (minimum audience size of 1,000 users)."
  },
  {
    "title": "Google Analytics 360 Audience Targeting",
    "description": "Shares GA360 remarketing lists based on site/app behavior for targeting or building similar audiences."
  },
  {
    "title": "Demographics Targeting",
    "description": "Sets up ad targeting based on demographics like gender, age, parental status, and household income."
  },
  {
    "title": "Affinity Targeting",
    "description": "Targets users with a demonstrated interest in a specific topic."
  },
  {
    "title": "In-market Targeting",
    "description": "Targets users actively researching or comparing related products and services."
  },
  {
    "title": "Custom Audience Targeting",
    "description": "Reaches audiences based on keywords, URLs, and apps related to your product or service."
  },
  {
    "title": "Life Events Targeting",
    "description": "Reaches audiences during key life events like moving, graduating, getting married, or having a baby."
  },
  {
    "title": "Geography Targeting",
    "description": "Targets by region (states, cities, postcodes), or specific locations (business chains, POIs, street addresses, coordinates)."
  },
  {
    "title": "Day and Time Targeting",
    "description": "Specifies serving ads by days and times in user or advertiser timezones."
  },
  {
    "title": "Similar Audiences",
    "description": "Expands existing audiences by targeting users with similar behavior and interests."
  },
  {
    "title": "Third-party (DMP) Audiences",
    "description": "Syncs audiences from third-party Data Management Platforms (DMPs) for more granular targeting."
  }
]

table = '\n'.join(f'{method["title"]}: {method["description"]}' for method in methods)

for avatar in avatars:
    print('Running:', avatar['persona'])

    #preserve tokens reset history
    chat._history = history

    prompt = f'Pretend you are a marketing expert, you are selling {product} to persona {avatar["persona"]}. Pick 5 audiences that will generate the most sales from the following JSON list of dictionaries:\n\n{table}'
    response = chat.send_message(
      f'Format your response as a JSON formatted list of dictionaries with quoted keys: title, description.\n\n{prompt}'
    )
    avatar['targeting'] = response_to_list(response)
    print(json.dumps(avatar['targeting'], indent=2))


Step 6: Deep Funnel Analysis

Use the AI to get deep funnel insights such as objections that could be addressed by the creative or the landing page

In [ ]:

for avatar in avatars:
  print('Running:', avatar['persona'])

  # preserve tokens reset history
  chat._history = history

  prompt = f'Pretend you are {avatar["persona"]}. What is the most likely question you will ask about the {product} shown in the attached image.'
  response = model_vision.generate_content(
    contents=[f'Format your response as a plain text question without quotes.\n\n{prompt}', target_picture]
  )
  avatar['question'] = response.text.strip()

  prompt = f'What response would a good sales person give to {avatar["persona"]} asking a qestion about {product}. The question is:\n\n {avatar["question"]}'
  response = chat.send_message(
    f'Format your response as a plain text sentence without quotes.\n\n{prompt}'
  )
  avatar['response'] = response.text.strip()
  print(avatar['question'])
  print(avatar['response'])

Step 7: Generate A Landing page for Each Avatar

This is usually done with barnd inputs and a template but we can rely on Ai to get us close. You can use JSON to feed into an existing template

In [ ]:

for avatar in avatars:
  print('Running:', avatar['persona'])

  # preserve tokens reset history
  chat._history = history

  prompt = f'Give me an example of a highly converting landing page for {avatar["persona"]} who want to buy a {product}.  The landing page should include: Hero Section, Benefits, Social Proof, Features List, Address Objection With Response, and Call To Action. The objection to address is: {avatar["question"]}. The response is: {avatar["response"]}. Rephrase the obection to be more engaging to the persona.'
  response = chat.send_message(
    f'Format your response as a JSON dictionary.\n\n{prompt}'
  )
  avatar['page'] = response_to_dict(response)
  print(json.dumps(avatar['page'], indent=2))